In [1]:
!pip install casadi
from IPython.display import clear_output
import os

import numpy as np
import casadi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

os.makedirs('images',exist_ok=True)
if os.name == 'nt':
    plt.rcParams['font.family'] = 'MS Gothic'
elif os.name == 'posix':
    !pip install japanize-matplotlib


clear_output()

In [2]:
alpha = 30.828
beta = 86.688
delta = 3.522*10**-4
gamma = 0.1
Tin = 104.9
cin = 5.1*10**3
k10 = 1.287*10**12
k20 = 9.043*10**6
E1 = 9758.3
E2 = 8560.0
HAB = 4.2
HBC = -11.0
HAD = -41.85

nx = 4 #状態変数の次元
nu = 2 #制御変数の次元
nw = nx #プロセスノイズの次元
ny = 2 #観測状態の次元
nv = ny

# コスト関数の重み
Q_MPC = casadi.diag([0.2,1,0.5,0.2])
Q_f_MPC = casadi.diag([0.2,1,0.5,0.2])
R_MPC = casadi.diag([0.05,5e-3])

Q_MHE = casadi.diag([10,10,10,10])
Q_i_MHE = casadi.diag([0.1,0.1,0.1,0.1]) * 0
R_MHE = casadi.diag([10,10])

# 予測ホライズン等
T = 1/3
K = 40
dt = T/K

# 制約条件
x_lb = [-np.inf,-np.inf,-np.inf,-np.inf]
x_ub = [np.inf,np.inf,np.inf,np.inf]
u_lb = [3,-9000]
u_ub = [35,0]
w_lb = [-np.inf,-np.inf,-np.inf,-np.inf]
w_ub = [np.inf,np.inf,np.inf,np.inf]
v_lb = [-np.inf,-np.inf]
v_ub = [np.inf,np.inf]

# 目標
x_ref = casadi.DM([ 1.37e3, 0.95e3, 110.0, 108.6])
u_ref = casadi.DM([ 5.0, -1190.0])

total_MPC = nx*(K+1) + nu*K
total_MHE = nx*(K+1) + nw*K + nu*K + ny*K + nv*K + nx

In [3]:
def make_f():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)
    noises = casadi.SX.sym("noises",nw)

    Ca = states[0]
    Cb = states[1]
    TR = states[2]
    TC = states[3]
    u1 = ctrls[0]
    u2 = ctrls[1]

    k1 = k10*casadi.exp(-E1/(TR+273.15))
    k2 = k20*casadi.exp(-E2/(TR+273.15))

    Ca_dot = -k1*Ca - k2*Ca**2 +(cin - Ca)*u1
    Cb_dot = k1*Ca - k1*Cb - Cb*u1
    TR_dot = -delta*(k1*Ca*HAB + k1*Cb*HBC + k2*Ca**2*HAD) + alpha*(TC-TR) + (Tin - TR)*u1
    TC_dot = beta*(TR-TC) + gamma*u2

    states_dot = casadi.vertcat(Ca_dot,Cb_dot,TR_dot,TC_dot) + noises

    f = casadi.Function("f",[states,ctrls,noises],[states_dot],['x','u',"w"],['x_dot'])
    return f

def make_RK4():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)
    noises = casadi.SX.sym("noises",nw)

    f = make_f()

    k1 = f(x=states,u=ctrls,w=noises)["x_dot"]
    k2 = f(x=states+dt*k1/2,u=ctrls,w=noises)["x_dot"]
    k3 = f(x=states+dt*k2/2,u=ctrls,w=noises)["x_dot"]
    k4 = f(x=states+dt*k3,u=ctrls,w=noises)["x_dot"]

    states_next = states + dt*(k1+2*k2+2*k3+k4)/6

    RK4 = casadi.Function("RK4",[states,ctrls,noises],[states_next],["x","u","w"],["x_next"])
    return RK4

def make_h():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)
    onoises = casadi.SX.sym("onoises",nv)

    C = casadi.DM([
        [0,0,1,0],
        [0,0,0,1]
    ])

    observes = C @ states + onoises

    h = casadi.Function("h",[states,ctrls,onoises],[observes],['x',"u","v"],['y'])
    return h

def make_integrator():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)
    noises = casadi.SX.sym("noises",nw)

    f = make_f()
    ode = f(x=states, u=ctrls,w=noises)["x_dot"]
    params = casadi.vertcat(ctrls,noises)
    dae = {"x":states,"p":params,"ode":ode}

    I = casadi.integrator("I","cvodes",dae,0,dt)
    return I

def compute_MPC_stage_cost(x,u):
    x_diff = x - x_ref
    u_diff = u - u_ref
    cost = (casadi.dot(Q_MPC@x_diff,x_diff) + casadi.dot(R_MPC@u_diff,u_diff)) / 2
    return cost

def compute_MPC_final_cost(x):
    x_diff = x - x_ref
    cost = casadi.dot(Q_f_MPC@x_diff,x_diff) / 2
    return cost

def compute_MHE_stage_cost(w,v):
    cost = (casadi.dot(Q_MHE@w,w) + casadi.dot(R_MHE@v,v)) / 2
    return cost

def compute_MHE_initial_cost(x_diff):
    cost = casadi.dot(Q_i_MHE@x_diff,x_diff) / 2
    return cost

In [4]:
def make_MPC_nlp():
    RK4 = make_RK4()

    U = [casadi.SX.sym(f"u_{i}",nu) for i in range(K)]
    X = [casadi.SX.sym(f"x_{i}",nx) for i in range(K+1)]
    G = []

    J = 0

    for k in range(K):
        J += compute_MPC_stage_cost(X[k],U[k]) * dt
        eq = X[k+1] - RK4(x=X[k],u=U[k])["x_next"]
        G.append(eq)
    J += compute_MPC_final_cost(X[-1])

    option = {'print_time':False,'ipopt':{'max_iter':1000,'print_level':5}}
    nlp = {"x":casadi.vertcat(*X,*U),"f":J,"g":casadi.vertcat(*G)}
    S = casadi.nlpsol("S_MPC","ipopt",nlp,option)
    return S

def make_MHE_nlp():
    RK4 = make_RK4()
    h = make_h()

    X = [casadi.SX.sym(f"x_{i}",nx) for i in range(K+1)]
    U = [casadi.SX.sym(f"u_{i}",nu) for i in range(K)]
    Y = [casadi.SX.sym(f"y_{i}",ny) for i in range(K)]
    W = [casadi.SX.sym(f"w_{i}",nw) for i in range(K)]
    V = [casadi.SX.sym(f"v_{i}",nv) for i in range(K)]
    x_hat = casadi.SX.sym("x_hat",nx)
    G = []

    J = 0
    x_diff = X[0] - x_hat
    J += compute_MHE_initial_cost(x_diff)
    for k in range(K):
        J += compute_MHE_stage_cost(W[k],V[k]) * dt
        state_eq = X[k+1] - RK4(x=X[k],u=U[k],w=W[k])["x_next"]
        obs_eq = Y[k] - h(x=X[k],u=U[k],v=V[k])["y"]
        G.append(state_eq)
        G.append(obs_eq)

    option = {'print_time':False,'ipopt':{'max_iter':1000,'print_level':5}}
    nlp = {"x":casadi.vertcat(*X,*W,*U,*Y,*V,x_hat),"f":J,"g":casadi.vertcat(*G)}
    S = casadi.nlpsol("S_MHE","ipopt",nlp,option)
    return S

In [5]:
def compute_optimal_control(S,x_s,x0):
    x_s = x_s.full().ravel().tolist()
    
    lbx = x_s + x_lb*K + u_lb*K
    ubx = x_s + x_ub*K + u_ub*K
    lbg = [0]*nx*K
    ubg = [0]*nx*K

    res = S(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0)
    
    offset = nx*(K+1)
    x0 = res["x"]
    u_opt = x0[offset:offset+nu]
    return u_opt, x0

def compute_optimal_estimation(S,us,ys,x0,x_hat):
    lbx = x_lb*(K+1) + w_lb*K + us + ys + v_lb*K + x_hat
    ubx = x_ub*(K+1) + w_ub*K + us + ys + v_ub*K + x_hat
    lbg = ([0]*nx + [0]*ny)*K
    ubg = ([0]*nx + [0]*ny)*K

    res = S(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0)
    
    # offset = nx*(N+1)
    x0 = res["x"]
    return x0

In [6]:
S_MPC = make_MPC_nlp()
S_MHE = make_MHE_nlp()

In [ ]:
I = make_integrator()
h = make_h()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)

x_init = casadi.DM([2.02e3, 1.07e3, 100.0, 97.1]) # 初期値
x0 = casadi.DM.zeros(total_MPC)

X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    u_opt,x0 = compute_optimal_control(S_MPC,x_current,x0)
    p_current = casadi.vertcat(u_opt,casadi.DM.zeros(nw))
    x_current = I(x0=x_current,p=p_current)["xf"]
    X.append(x_current)
    U.append(u_opt)

X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

for k in range(nx):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
    plt.show()

for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
    plt.show()

# plt.figure(figsize=(12,4))

# plt.subplot(1,3,1)
# for k in range(2):
#     plt.plot(t_eval,X[:,k],label=f"x_{k}")
# plt.legend()

# plt.subplot(1,3,2)
# for k in range(2,4):
#     plt.plot(t_eval,X[:,k],label=f"x_{k}")
# plt.legend()

# plt.subplot(1,3,3)
# for k in range(nu):
#     plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
# plt.legend()

# plt.savefig("images/chap8_cart_pole_MPC.png")
# plt.show()

In [ ]:
t_span = [0,10]
t_eval = np.arange(*t_span,dt)

I = make_integrator()
h = make_h()

x_init = casadi.DM([2.02e3, 1.07e3, 100.0, 97.1]) # 初期値
# u_init = casadi.DM.zeros(nu)
# y_init = h(x=x_init,u=u_init)["y"]

x_hat = casadi.DM([2.02e3+100, 1.07e3+100, 100.0+5, 97.1-7])
# x0 = make_x0(x_hat)
x0 = casadi.DM.zeros(total_MHE)

X = [x_init]
X_est = [x_hat]
U = []
Y = []
x_current = x_init
for t in t_eval:
    if len(Y) >= K:
        ys = casadi.vertcat(*Y[-K:]).full().ravel().tolist()
        us = casadi.vertcat(*U[-K:]).full().ravel().tolist()
        x_hat_list = x_hat.full().ravel().tolist()
        x0 = compute_optimal_estimation(S_MHE,us,ys,x0,x_hat_list)
        x_hat = x0[:nx]
        X_est.append(x_hat)
    u_current = np.array([ 5.0, -1190.0]) + np.sin(t)
    w_current = np.random.normal(scale=1e-4,size=(nw,))
    v_current = np.random.normal(scale=1e-4,size=(nv,))
    p_current = casadi.DM([*u_current,*w_current])
    y_current = h(x=x_current,u=u_current,v=v_current)["y"]
    x_current = I(x0=x_current,p=p_current)["xf"]
    X.append(x_current)
    Y.append(y_current)
    U.append(u_current)

X.pop()
X = np.array(X).reshape(t_eval.size,nx)
X_est.pop()
X_est = np.array(X_est).reshape(-1,nx)

plt.figure(figsize=(12,4))

for i in range(nx):
    plt.subplot(1,nx,i+1)
    plt.plot(t_eval[:-K],X[:-K,i],label=f"x_{i}")
    plt.step(t_eval[:-K],X_est[:,i],linestyle="--",label=f"x_{i}_estimated")
    plt.legend()

# plt.savefig("images/chap8_msd_MHE.png")
plt.show()

In [7]:
t_span = [0,1]
t_eval = np.arange(*t_span,dt)

I = make_integrator()
h = make_h()

x_init = casadi.DM([2.02e3, 1.07e3, 100.0, 97.1]) # 初期値
# u_init = casadi.DM.zeros(nu)
# y_init = h(x=x_init,u=u_init)["y"]

x_hat = casadi.DM([2.02e3+100, 1.07e3+100, 100.0+5, 97.1-7])
x0_MPC = casadi.DM.zeros(total_MPC)
x0_MHE = casadi.DM.zeros(total_MHE)

X = [x_init]
X_est = [x_hat]
U = []
Y = []
x_current = x_init
x_current_est = x_hat
for t in t_eval:
    if len(Y) >= K:
        ys = casadi.vertcat(*Y[-K:]).full().ravel().tolist()
        us = casadi.vertcat(*U[-K:]).full().ravel().tolist()
        x_hat_list = x_hat.full().ravel().tolist()
        x0_MHE = compute_optimal_estimation(S_MHE,us,ys,x0_MHE,x_hat_list)
        x_hat = x0_MHE[:nx]
        X_est.append(x_hat)
        x_current_est = x0_MHE[nx*K:nx*(K+1)]
    u_current,x0_MPC = compute_optimal_control(S_MPC,x_current_est,x0_MPC)
    w_current = casadi.DM(np.random.normal(scale=1e-4,size=(nw,)))
    v_current = np.random.normal(scale=1e-4,size=(nv,))
    p_current = casadi.vertcat(u_current,w_current)
    y_current = h(x=x_current,u=u_current,v=v_current)["y"]
    x_current = I(x0=x_current,p=p_current)["xf"]
    X.append(x_current)
    Y.append(y_current)
    U.append(u_current)

X.pop()
X = np.array(X).reshape(t_eval.size,nx)
X_est.pop()
X_est = np.array(X_est).reshape(-1,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

for i in range(nx):
    plt.subplot(1,nx,i+1)
    plt.plot(t_eval[:-K],X[:-K,i],label=f"x_{i}")
    plt.step(t_eval[:-K],X_est[:,i],linestyle="--",label=f"x_{i}_estimated")
    plt.legend()
plt.show()

for i in range(nu):
    plt.subplot(nu,1,i+1)
    plt.step(t_eval[:-K],U[:-K,i],label=f"u_{i}")
    plt.legend()

# plt.savefig("images/chap8_msd_MHE.png")
plt.show()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1104
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      826

Total number of variables............................:      240
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       80
                     variables with only upper bounds:        0
Total number of equality constraints.................:      160
Total number of inequality c

CasADi - 2024-01-23 19:18:37 WARNING("S_MHE:nlp_g failed: Inf detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


NameError: name 'x0' is not defined

In [ ]:
t_span = [0,20]
t_eval = np.arange(*t_span,dt)

x_init = casadi.DM([0,np.pi,0,0]) # 初期値
y_init = h(x_init)
x0_MPC = casadi.DM.zeros(total_MPC)
x0_MHE = casadi.DM.zeros(total_MHE)
x_hat = casadi.DM([0,0,0,0])

X = [x_init]
X_est = []
Y = [y_init]
U = []
x_current = x_init
x_current_est = x_hat
for t in t_eval:
    if len(Y) > K:
        ys = casadi.vertcat(*Y[-K:]).full().ravel().tolist()
        us = casadi.vertcat(*U[-K:]).full().ravel().tolist()
        x_hat_list = x_hat.full().ravel().tolist()
        x0 = compute_optimal_estimation(S_MHE,us,ys,x0_MHE,x_hat_list)
        x_est = x0[:nx]
        X_est.append(x_est)
        x_hat = x_est
        x_current_est = x0[nx*K:nx*(K+1)]
    u_opt,x0_MPC = compute_optimal_control(S_MPC,x_current_est,x0_MPC)
    w_current = np.random.normal(size=(nw,),scale=0.1)
    p_current = casadi.DM([u_opt,*w_current])
    x_current = I(x0=x_current,p=p_current)["xf"]
    y_current = h(x_current)
    X.append(x_current)
    Y.append(y_current)
    U.append(u_opt)
    # U.append(u_opt)

X.pop()
X = np.array(X).reshape(t_eval.size,nx)
# X_est.pop()
X_est = np.array(X_est).reshape(-1,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,6))

for i in range(nx):
    plt.subplot(2,2,i+1)
    plt.plot(t_eval[:-K],X[:-K,i],label=f"x_{i}")
    plt.step(t_eval[:-K],X_est[:,i],linestyle="--",label=f"x_{i}_estimated")
    plt.legend()

plt.savefig("images/chap8_cart_pole_MPCxMHE_state.png")
plt.show()

plt.figure(figsize=(12,4))

for i in range(nu):
    plt.subplot(nu,1,i+1)
    plt.step(t_eval[:-K],U[:-K,i],label=f"u_{i}")
    plt.legend()

plt.savefig("images/chap8_cart_pole_MPCxMHE_ctrl.png")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
frames = np.arange(0,t_eval.size)
fps = 1 / dt

def update_figure(i):
    x_lim_min = -4
    x_lim_max = 4
    y_lim_min = -2
    y_lim_max = 2
    u_scale = 15

    ax.cla()
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.set_ylim(y_lim_min, y_lim_max)
    ax.set_aspect("equal")

    x,theta,_,_ = X[i]
    u, = U[i]

    points = np.array([
        [x,x-l*np.sin(theta)],
        [0,l*np.cos(theta)]
    ])

    ax.hlines(0,x_lim_min,x_lim_max,colors="black")
    ax.scatter(*points,color="blue", s=50)
    ax.plot(*points, color='blue', lw=2)
    ax.arrow(x,0,u/u_scale,0,width=0.02,head_width=0.06,head_length=0.12,length_includes_head=False,color="green",zorder=3)

    w = 0.2
    h = 0.1
    rect = patches.Rectangle(xy=(x-w/2,-h/2), width=w, height=h,color="black")
    ax.add_patch(rect)

ani = FuncAnimation(fig, update_figure, frames=frames)
ani.save("images/chap8_cart_pole.gif",writer="pillow",fps=fps)

In [ ]:
# import os
# import shutil
# import glob
# import natsort
# import cv2
# shutil.rmtree("./tmp",ignore_errors=True)
# os.mkdir("./tmp")
# for i in np.linspace(0,149,12):
#     i = int(i)
#     update_figure(i)
#     fig.savefig(f"./tmp/_frame_{i}.png")
# paths = glob.glob("./tmp/*.png")
# paths = natsort.natsorted(paths)
# paths = [cv2.imread(path) for path in paths]
# paths = np.stack(paths,axis=0)
# paths = paths.reshape(4,3,600,1200,3)
# paths = np.concatenate(paths,axis=1)
# paths = np.concatenate(paths,axis=1)
# cv2.imwrite("images/chap8_cart_pole_anim.png",paths)
# shutil.rmtree("./tmp",ignore_errors=True)